In [20]:
# Get all links to the detail page from digilib.uns.ac.id
# import requests
# from bs4 import BeautifulSoup as bs
# import pyquery
# import time
# import json
# import concurrent.futures

# links = []

# abstracts_data = []

# pages = 1809

# MAX_THREADS = 50

# def get_link(page):
#   res = requests.get(f'https://digilib.uns.ac.id/dokumen/fakultas/7/Fak-KIP/{page}')
#   html_page = bs(res.content, 'html.parser')
#   document_cards = html_page.select(
#       '#digilib-body > div > div > div.col-md-8 > div.mb-5')
#   for card in document_cards:
#     document_type = card.select(
#         '.dokumen-search-body .detail div:nth-child(2)')[0].text.strip().lower()
#     if(document_type == 'skripsi'):
#       anchor = card.find('a')
#       if 'https://digilib.uns.ac.id/dokumen/detail/' in anchor.get('href'):
#           links.append(anchor['href'])

# def get_detail_links():
#   with concurrent.futures.ThreadPoolExecutor(max_workers=MAX_THREADS) as executor:
#       executor.map(get_link, range(1, pages + 1))
#   with open('detail_links.json', 'w') as output:
#     output.write(json.dumps(links, indent=4, sort_keys=True))



In [21]:
# # get abstract and title from detail page
# def get_abstract_and_title(link):
  
#   res = requests.get(link)
#   html_page = bs(res.content, 'html.parser')
#   table = html_page.find('table', {'class': 'table table-responsive'})
#   table_trs = table.find_all('tr')
#   nim = table_trs[2].text.strip()
#   if 'K35' in nim or 'K.35' in nim or 'K. 35' in nim or 'K 35' in nim:
#     print('==============================')
#     print(
#         f'Getting abstract from link: {link} \n')
#     tdabstrak = table_trs[13].find_all('td')
#     tdjudul = table_trs[4].find_all('td')
#     abstract = tdabstrak[2].get_text()
#     title = tdjudul[2].get_text()

#     abstracts_data.append({
#         'title': title,
#         'abstract': abstract,
#     })
#   else:
#     print('==============================')
#     print(
#         f'Link skipped: {link} \n')
#     time.sleep(1)


# def add_id_to_abstract(i):
#   abstracts_data[i]['id'] = i + 1


# def get_abstracts():
#   with open('detail_links.json', 'r') as input:
#     links_from_json = json.load(input)
#     with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
#         executor.map(get_abstract_and_title, (link for link in links_from_json))
#     with concurrent.futures.ThreadPoolExecutor(max_workers=MAX_THREADS) as executor:
#         executor.map(add_id_to_abstract,
#                      (i for i in range(len(abstracts_data))))
#     with open('abstracts_data.json', 'w') as output:
#       output.write(json.dumps(abstracts_data, indent=4, sort_keys=True))


# get_abstracts()

In [22]:
import progressbar

def progress_bar():
    return progressbar.ProgressBar(maxval=78, widgets=[
        ' [', progressbar.Timer(), '] ',
        progressbar.Bar(marker='0', left='[', right=']'),
        ' (', progressbar.ETA(), ') ',
    ])

In [23]:
# case folding
import json
import re
import time
import sys
import concurrent.futures
import progressbar


processed = []
count = 0
with open('data/abstracts_data.json') as abstracts_json:
  abstracts = json.load(abstracts_json)

bar = progress_bar()
bar.start()

for i, abstract in enumerate(abstracts):
  bar.update(i + 1)
  answer = re.sub('[^a-z]+', ' ', abstract['abstract'].casefold())
  abstracts[i]['abstract'] = answer

with open('data/case_folded.json', 'w') as outfile:
	outfile.write(json.dumps(abstracts, sort_keys=True, indent=4))
  
bar.finish()
print('Case folding done')

Case folding done


In [24]:
# translate if english
import json
from langdetect import detect
import progressbar
import translators as ts

with open('data/case_folded.json') as case_folded_json:
  abstracts = json.load(case_folded_json)

bar = progress_bar().start()

for i, abstract in enumerate(abstracts):
  bar.update(i + 1)
  detector = detect(abstract['abstract'])
  if detector == 'en':
    translation = ts.google(abstract['abstract'], from_language='en', to_language='id')
    abstracts[i]['abstract'] = translation


with open('data/translated.json', 'w') as outfile:
  outfile.write(json.dumps(abstracts, sort_keys=True, indent=4))
bar.finish()

 [Elapsed Time: 0:00:02] [00000000000000000000000000000000000] (Time: 0:00:02) 


In [25]:
# case folding again
import json
import re
import time
import sys
import concurrent.futures
import progressbar


processed = []
count = 0
with open('data/translated.json') as abstracts_json:
  abstracts = json.load(abstracts_json)

bar = progress_bar().start()

for i, abstract in enumerate(abstracts):
  bar.update(i + 1)
  answer = re.sub('[^a-z]+', ' ', abstract['abstract'].casefold())
  abstracts[i]['abstract'] = answer

with open('data/case_folded.json', 'w') as outfile:
	outfile.write(json.dumps(abstracts, sort_keys=True, indent=4))

bar.finish()
print('Case folding done')


Case folding done


In [26]:
# stemming
import json
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import progressbar

with open('data/case_folded.json') as case_folded_json:
  abstracts = json.load(case_folded_json)

factory = StemmerFactory()
stemmer = factory.create_stemmer()


bar = progress_bar()
bar.start()

for i, abstract in enumerate(abstracts):
  bar.update(i + 1)
  stemmed = stemmer.stem(abstract['abstract'])
  abstracts[i]['abstract'] = stemmed


with open('data/stemmed_abstracts.json', 'w') as outfile:
  outfile.write(json.dumps(abstracts, sort_keys=True, indent=4))

bar.finish()

 [Elapsed Time: 0:00:41] [00000000000000000000000000000000000] (Time: 0:00:41) 
